# 设计一个具有1个循环单元的循环神经网络（RNN）

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 定义循环神经网络
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)
        return out

# 实例化模型
input_size = 28
hidden_size = 128
output_size = 10
model = RNN(input_size, hidden_size, output_size)
print(model)

# 构造循环神经网络的前向传播和后向传播程序

In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 定义前向传播和后向传播过程
def train(model, trainloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.view(-1, 28, 28)  # 重新调整输入的形状
            optimizer.zero_grad()  # 清除梯度
            outputs = model(inputs)  # 前向传播
            loss = criterion(outputs, labels)  # 计算损失
            loss.backward()  # 后向传播
            optimizer.step()  # 更新参数

            running_loss += loss.item()
            if i % 200 == 199:  # 每200个batch打印一次损失
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200:.3f}")
                running_loss = 0.0
    print('Finished Training')

# 训练循环神经网络

In [ ]:
# 加载 MNIST 数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# 训练模型
train(model, trainloader, criterion, optimizer, epochs=5)

# 利用循环神经网络实现 MNIST 手写体识别，并与卷积神经网络、多层感知器结果进行比较

In [ ]:
import numpy as np

# 测试循环神经网络
def test(model, testloader):
    model.eval()  # 设置为评估模式
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.view(-1, 28, 28)  # 重新调整输入的形状
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# 测试模型
test(model, testloader)

# 可视化一些测试结果
dataiter = iter(testloader)
images, labels = next(dataiter)  # 使用 next 函数
images = images.view(-1, 28, 28)  # 重新调整输入的形状
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# 显示图片
def imshow(img):
    img = img / 2 + 0.5  # 反归一化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 打印前5个图像及其预测标签
imshow(torchvision.utils.make_grid(images[:5].view(-1, 1, 28, 28)))
print('GroundTruth: ', ' '.join(f'{labels[j]}' for j in range(5)))
print('Predicted: ', ' '.join(f'{predicted[j]}' for j in range(5)))